In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('~/playground/Expenses/2022.xlsx', sheet_name="USD")  
df = df.drop(columns=["Notes", "Description"])

In [ ]:
df = df[df["Transaction Type"] == "debit"]

In [ ]:
df['Amount'].sum()

In [ ]:
df = df.assign(parsed=df['Category'].str.split('+')).explode('parsed')

In [ ]:
df = df.drop(columns="Category")

In [ ]:
df

In [ ]:
def category(parsed):
    return parsed.split('(').str[0].str.strip()

In [ ]:
def amount(parsed):
    return parsed.split('(').str[1]

In [ ]:
df = df.assign(Category=lambda x: category(x.parsed.str))

In [ ]:
df = df.assign(RAmount=lambda x: amount(x.parsed.str))

In [ ]:
df['RAmount'] = df['RAmount'].str.split(')').str[0].astype('float')

In [ ]:
df['RAmount'].fillna(df['Amount'], inplace=True)

In [ ]:
df = df.drop(columns='parsed')

In [ ]:
df['Date'] =  pd.DatetimeIndex(df.Date)

In [ ]:
monthly_report = df.groupby([df.Date.dt.to_period('M'), "Category"])['RAmount'].sum().reset_index(name='Amount').sort_values(by=['Date', 'Amount'], ascending=False)
monthly_report['Amount'].sum()

In [ ]:
pv_table = pd.pivot_table(monthly_report, index = 'Category', columns = 'Date', values = 'Amount')
fig = pv_table.plot(kind = 'bar', figsize=(20,10),  yticks=(100, 250, 500, 1000, 1500, 2000, 2500, 5000))

In [ ]:
#fig.get_figure().savefig("monthly_category_report.png", format="png")

In [ ]:
category_report = df.groupby(["Category"])['RAmount'].sum().reset_index(name='Amount').sort_values(by=['Amount'], ascending=False)
with pd.option_context("display.max_rows", 1000):
    print(category_report)

In [ ]:
category_report.plot.bar(x='Category', y='Amount', figsize=(20,10),  yticks=(100, 250, 500, 1000, 1500, 2000, 2500, 5000))

In [ ]:
category_report['Amount'].sum()